# iPSCs reprogramming

In [1]:
import anndata
import scvelo as scv
import pandas as pd
import numpy as np
import scanpy as scp
from scipy.sparse import issparse, csr_matrix, coo_matrix
from tqdm import tqdm
from os.path import join
import ocelli as oci

data_folder = '../../../../experiments/reprogramming/data'
output_folder = '../../../../experiments/reprogramming/output'

SEED = 17

In [2]:
%%time
adata = anndata.read_h5ad(join(data_folder, 'reprogramming_doublet.h5ad'))

obs = pd.DataFrame(index=list(adata.obs.index))
obs['timestamp'] = [el.split('_')[0][1:] for el in adata.obs['origin']]
obs['origin'] = [el.split('_')[1] for el in adata.obs['origin']]
obs['doublet_score'] = list(adata.obs['doublet_score'])

adata.obs = obs
adata.var = pd.DataFrame(index=list(anndata.read_h5ad(join(data_folder, 'Pancreas/endocrinogenesis_day15.h5ad'))[0].var.index))
adata.layers = {}

adata = adata[adata.obs['origin'].isin(['Dox', 'serum']), :]

adata = adata[adata.obs['doublet_score'] < 0.3]
scp.pp.filter_cells(adata, min_counts=2000)
scp.pp.filter_genes(adata, min_cells=50)
scp.pp.downsample_counts(adata, counts_per_cell=15000, random_state=SEED)

adata.write(join(output_folder, 'R1.h5ad'))

CPU times: user 57.3 s, sys: 12.3 s, total: 1min 9s
Wall time: 1min 9s


In [2]:
%%time
adata = anndata.read_h5ad(join(output_folder, 'R1.h5ad'))
oci.pp.LDA(adata, n_components=20, output_key='lda', verbose=1, max_iter=30, random_state=SEED)
oci.pp.modality_generation(adata, topic_key='lda', norm_log=True, verbose=True)
adata.X = csr_matrix(([], ([], [])), shape=adata.shape)
adata.write(join(output_folder, 'R2.h5ad'))

iteration: 1 of max_iter: 30
iteration: 2 of max_iter: 30
iteration: 3 of max_iter: 30
iteration: 4 of max_iter: 30
iteration: 5 of max_iter: 30
iteration: 6 of max_iter: 30
iteration: 7 of max_iter: 30
iteration: 8 of max_iter: 30
iteration: 9 of max_iter: 30
iteration: 10 of max_iter: 30
iteration: 11 of max_iter: 30
iteration: 12 of max_iter: 30
iteration: 13 of max_iter: 30
iteration: 14 of max_iter: 30
iteration: 15 of max_iter: 30
iteration: 16 of max_iter: 30
iteration: 17 of max_iter: 30
iteration: 18 of max_iter: 30
iteration: 19 of max_iter: 30
iteration: 20 of max_iter: 30
iteration: 21 of max_iter: 30
iteration: 22 of max_iter: 30
iteration: 23 of max_iter: 30
iteration: 24 of max_iter: 30
iteration: 25 of max_iter: 30
iteration: 26 of max_iter: 30
iteration: 27 of max_iter: 30
iteration: 28 of max_iter: 30
iteration: 29 of max_iter: 30
iteration: 30 of max_iter: 30
Modality 0 --> saved to adata.obsm[modality0].
Modality 1 --> saved to adata.obsm[modality1].
Modality 2 --> 

In [8]:
%%time
adata = anndata.read_h5ad(join(output_folder, 'R2.h5ad'))

paths = ['serum_0.0_0.5.h5ad', 'serum_0.5_1.0.h5ad', 'serum_1.0_1.5.h5ad', 
         'serum_1.5_2.0.h5ad', 'serum_2.0_2.5.h5ad', 'serum_2.5_3.0.h5ad', 
         'serum_3.0_3.5.h5ad', 'serum_3.5_4.0.h5ad', 'serum_4.0_4.5.h5ad', 
         'serum_4.5_5.0.h5ad', 'serum_5.0_5.5.h5ad', 'serum_5.5_6.0.h5ad', 
         'serum_6.0_6.5.h5ad', 'serum_6.5_7.0.h5ad', 'serum_7.0_7.5.h5ad', 
         'serum_7.5_8.0.h5ad', 'serum_8.0_8.25.h5ad', 'serum_8.25_8.5.h5ad', 
         'serum_8.5_8.75.h5ad', 'serum_8.75_9.0.h5ad', 'serum_9.0_9.5.h5ad', 
         'serum_9.5_10.0.h5ad', 'serum_10.0_10.5.h5ad', 'serum_10.5_11.0.h5ad', 
         'serum_11.0_11.5.h5ad', 'serum_11.5_12.0.h5ad', 'serum_12.0_12.5.h5ad', 
         'serum_12.5_13.0.h5ad', 'serum_13.0_13.5.h5ad', 'serum_13.5_14.0.h5ad', 
         'serum_14.0_14.5.h5ad', 'serum_14.5_15.0.h5ad', 'serum_15.0_15.5.h5ad',
         'serum_15.5_16.0.h5ad', 'serum_16.0_16.5.h5ad', 'serum_16.5_17.0.h5ad', 
         'serum_17.0_17.5.h5ad', 'serum_17.5_18.0.h5ad']

def filter_cells(adata, x):
    obs_in, var_in = list(), list()
    
    for el in adata.obs.index:
        if el in x.obs.index:
            obs_in.append(el)
    for el in adata.obs.index:
        if el in x.var.index:
            var_in.append(el)
            
    return x[obs_in, var_in]
        
barcode_map = dict()
for i, barcode in enumerate(adata.obs.index):
    barcode_map[barcode] = i
    
M = coo_matrix(([], ([], [])), shape=(adata.shape[0], adata.shape[0])).tocsr()
    
for path in tqdm(paths):
    x = anndata.read_h5ad(join(data_folder, 'tmaps/{}'.format(path)))    
    x = filter_cells(adata, x)
    
    obs = [barcode_map[el] for el in x.obs.index]
    var = [barcode_map[el] for el in x.var.index]
    
    M[np.ix_(obs, var)] = csr_matrix(x.X)

adata.uns['velocity_graph'] = M

adata.write(join(output_folder, 'R3.h5ad'))

100%|██████████| 38/38 [12:36<00:00, 19.91s/it]


CPU times: user 11min 10s, sys: 1min 5s, total: 12min 15s
Wall time: 12min 43s


In [ ]:
%%time
adata = anndata.read_h5ad(join(output_folder, 'R3.h5ad'))
oci.pp.neighbors(adata, n_neighbors=20, verbose=True)
oci.tl.MDM(adata, weights_key='lda', n_components=20, random_state=SEED, verbose=True)
oci.pp.neighbors(adata, modalities=['X_mdm'], n_neighbors=100, verbose=True)
adata.write(join(output_folder, 'R4.h5ad'))

In [2]:
adata = anndata.read_h5ad(join(output_folder, 'R4.h5ad'))

In [3]:
l = list()
for t in adata.obs['timestamp']:
    try:
        l.append(float(t))
    except:
        l.append(20.)

adata.obs['timestamp_float'] = l

In [4]:
%%time
oci.tl.timestamp_graph(adata, transitions_key='velocity_graph', timestamps_key='timestamp_float', n_edges=10, neighbors_key='X_mdm', verbose=True)

40it [04:16,  6.41s/it]

Timestamp-based graph constructed.
CPU times: user 4min 38s, sys: 3min 9s, total: 7min 48s
Wall time: 5min 33s


In [6]:
%%time
oci.tl.FA2(adata, n_components=3, n_iter=10000, random_state=SEED, output_key='X_fa2')

Oct 06, 2022 3:17:01 PM org.netbeans.modules.masterfs.watcher.Watcher getNotifierForPlatform
INFO: Native file watcher is disabled
Oct 06, 2022 3:17:07 PM org.gephi.io.processor.plugin.DefaultProcessor process
INFO: # Nodes loaded: 162,917 (162,917 added)
Oct 06, 2022 3:17:07 PM org.gephi.io.processor.plugin.DefaultProcessor process
INFO: # Edges loaded: 1,629,170 (1,479,011 added)


*

Exception in thread "main" java.util.concurrent.RejectedExecutionException: Task java.util.concurrent.FutureTask@1097a269[Not completed, task = java.util.concurrent.Executors$RunnableAdapter@6edd0b80[Wrapped task = kco.forceatlas2.ForceAtlas2$AttractionTask@72642fcd]] rejected from java.util.concurrent.ThreadPoolExecutor@7bc7ab76[Terminated, pool size = 0, active threads = 0, queued tasks = 0, completed tasks = 13602]
	at java.base/java.util.concurrent.ThreadPoolExecutor$AbortPolicy.rejectedExecution(ThreadPoolExecutor.java:2055)
	at java.base/java.util.concurrent.ThreadPoolExecutor.reject(ThreadPoolExecutor.java:825)
	at java.base/java.util.concurrent.ThreadPoolExecutor.execute(ThreadPoolExecutor.java:1355)
	at java.base/java.util.concurrent.AbstractExecutorService.submit(AbstractExecutorService.java:118)
	at kco.forceatlas2.ForceAtlas2.attraction(Unknown Source)
	at kco.forceatlas2.ForceAtlas2.goAlgo(Unknown Source)
	at kco.forceatlas2.Main.main(Unknown Source)


CPU times: user 483 ms, sys: 27.3 ms, total: 511 ms
Wall time: 1min 6s


In [8]:
adata.write(join(output_folder, 'R4.h5ad'))

In [9]:
1

1

In [2]:
adata = anndata.read_h5ad(join(output_folder, 'R-8+.h5ad'))

In [3]:
plot = oci.pl.scatter(adata, x_key='X_fa2', color_key='timestamp_float', cmap='gnuplot', marker_size=0.5, static=False)
plot.write_html(join(output_folder, '3d-8+.html'))

In [12]:
adata

AnnData object with n_obs × n_vars = 162917 × 17924
    obs: 'timestamp', 'origin', 'doublet_score', 'n_counts', 'timestamp_float', 'id'
    var: 'n_cells'
    uns: 'lda_params', 'modalities', 'vars_0', 'vars_1', 'vars_10', 'vars_11', 'vars_12', 'vars_13', 'vars_14', 'vars_15', 'vars_16', 'vars_17', 'vars_18', 'vars_19', 'vars_2', 'vars_3', 'vars_4', 'vars_5', 'vars_6', 'vars_7', 'vars_8', 'vars_9', 'velocity_graph'
    obsm: 'X_fa2', 'X_mdm', 'distances_X_mdm', 'distances_modality0', 'distances_modality1', 'distances_modality10', 'distances_modality11', 'distances_modality12', 'distances_modality13', 'distances_modality14', 'distances_modality15', 'distances_modality16', 'distances_modality17', 'distances_modality18', 'distances_modality19', 'distances_modality2', 'distances_modality3', 'distances_modality4', 'distances_modality5', 'distances_modality6', 'distances_modality7', 'distances_modality8', 'distances_modality9', 'graph', 'lda', 'modality0', 'modality1', 'modality10', 'modali

In [5]:
adata = anndata.read_h5ad(join(output_folder, 'R1.h5ad'))

In [6]:
l = list()
for t in adata.obs['timestamp']:
    try:
        l.append(float(t))
    except:
        l.append(20.)

adata.obs['timestamp_float'] = l

In [7]:
adata

AnnData object with n_obs × n_vars = 162917 × 17924
    obs: 'timestamp', 'origin', 'doublet_score', 'n_counts', 'timestamp_float'
    var: 'n_cells'

In [10]:
c = adata[adata.obs['timestamp_float'] < 20]
c = c[c.obs['timestamp_float'] > 8]
c

View of AnnData object with n_obs × n_vars = 65237 × 17924
    obs: 'timestamp', 'origin', 'doublet_score', 'n_counts', 'timestamp_float'
    var: 'n_cells'

In [4]:
adata = anndata.read_h5ad(join(output_folder, 'R1.h5ad'))

l = list()
for t in adata.obs['timestamp']:
    try:
        l.append(float(t))
    except:
        l.append(20.)

adata.obs['timestamp_float'] = l

adata = adata[adata.obs['timestamp_float'] < 20]
adata = adata[adata.obs['timestamp_float'] >= 8]

In [6]:
adata.obs

,timestamp,origin,doublet_score,n_counts,timestamp_float
D10.5_serum_C2_ACGAGCCAGAGATGAG-1,10.5,serum,0.125000,10455.0,10.5
D10.5_serum_C2_ACAGCTACACTCAGGC-1,10.5,serum,0.089888,9615.0,10.5
D10.5_serum_C2_AATCCAGGTTAAGAAC-1,10.5,serum,0.083110,3882.0,10.5
D10.5_serum_C2_AAGGAGCTCGCCAGCA-1,10.5,serum,0.181818,4517.0,10.5
D10.5_serum_C2_AACGTTGCAAGCCCAC-1,10.5,serum,0.083110,7167.0,10.5
...,...,...,...,...,...
D9_serum_C2_TTTGTCATCGGTCTAA-1,9,serum,0.097493,4101.0,9.0
D9_serum_C2_TTTGCGCTCTGGTGTA-1,9,serum,0.123377,8676.0,9.0
D9_serum_C2_TTTGGTTAGCGTTCCG-1,9,serum,0.145985,6397.0,9.0
D9_serum_C2_TTTGGTTCACATCCAA-1,9,serum,0.145985,4479.0,9.0


In [10]:
paths = ['serum_8.0_8.25.h5ad', 'serum_8.25_8.5.h5ad', 
         'serum_8.5_8.75.h5ad', 'serum_8.75_9.0.h5ad', 'serum_9.0_9.5.h5ad', 
         'serum_9.5_10.0.h5ad', 'serum_10.0_10.5.h5ad', 'serum_10.5_11.0.h5ad', 
         'serum_11.0_11.5.h5ad', 'serum_11.5_12.0.h5ad', 'serum_12.0_12.5.h5ad', 
         'serum_12.5_13.0.h5ad', 'serum_13.0_13.5.h5ad', 'serum_13.5_14.0.h5ad', 
         'serum_14.0_14.5.h5ad', 'serum_14.5_15.0.h5ad', 'serum_15.0_15.5.h5ad',
         'serum_15.5_16.0.h5ad', 'serum_16.0_16.5.h5ad', 'serum_16.5_17.0.h5ad', 
         'serum_17.0_17.5.h5ad', 'serum_17.5_18.0.h5ad']

def filter_cells(adata, x):
    obs_in, var_in = list(), list()
    
    for el in adata.obs.index:
        if el in x.obs.index:
            obs_in.append(el)
    for el in adata.obs.index:
        if el in x.var.index:
            var_in.append(el)
            
    return x[obs_in, var_in]
        
barcode_map = dict()
for i, barcode in enumerate(adata.obs.index):
    barcode_map[barcode] = i
    
M = coo_matrix(([], ([], [])), shape=(adata.shape[0], adata.shape[0])).tocsr()
    
for path in tqdm(paths):
    x = anndata.read_h5ad(join(data_folder, 'tmaps/{}'.format(path)))    
    x = filter_cells(adata, x)
    
    obs = [barcode_map[el] for el in x.obs.index]
    var = [barcode_map[el] for el in x.var.index]
    
    print(obs, var)
    
    M[np.ix_(obs, var)] = csr_matrix(x.X)

adata.uns['velocity_graph'] = M

  0%|          | 0/22 [00:00<?, ?it/s]

[50485, 50486, 50487, 50488, 50489, 50490, 50491, 50492, 50493, 50494, 50495, 50496, 50497, 50498, 50499, 50500, 50501, 50502, 50503, 50504, 50505, 50506, 50507, 50508, 50509, 50510, 50511, 50512, 50513, 50514, 50515, 50516, 50517, 50518, 50519, 50520, 50521, 50522, 50523, 50524, 50525, 50526, 50527, 50528, 50529, 50530, 50531, 50532, 50533, 50534, 50535, 50536, 50537, 50538, 50539, 50540, 50541, 50542, 50543, 50544, 50545, 50546, 50547, 50548, 50549, 50550, 50551, 50552, 50553, 50554, 50555, 50556, 50557, 50558, 50559, 50560, 50561, 50562, 50563, 50564, 50565, 50566, 50567, 50568, 50569, 50570, 50571, 50572, 50573, 50574, 50575, 50576, 50577, 50578, 50579, 50580, 50581, 50582, 50583, 50584, 50585, 50586, 50587, 50588, 50589, 50590, 50591, 50592, 50593, 50594, 50595, 50596, 50597, 50598, 50599, 50600, 50601, 50602, 50603, 50604, 50605, 50606, 50607, 50608, 50609, 50610, 50611, 50612, 50613, 50614, 50615, 50616, 50617, 50618, 50619, 50620, 50621, 50622, 50623, 50624, 50625, 50626, 50627

  5%|▍         | 1/22 [00:03<01:06,  3.17s/it]


KeyboardInterrupt: 